In [1]:
import polars as pl

# part1

In [2]:
reindeer_example = {
    "Comet": {
        "speed": 14,
        "max_travel_time": 10,
        "rest_time": 127
    },
    "Dancer": {
        "speed": 16,
        "max_travel_time": 11,
        "rest_time": 162
    }
}

In [3]:
day14_inputs = "inputs/day14.txt"
with open(day14_inputs, 'r') as file:
    day14_data = file.readlines()

In [4]:
reindeers = {}
for row in day14_data:
    data_split = row.strip().split(" ")
    
    reindeer_name = data_split[0]
    reindeers.setdefault(reindeer_name, {})["speed"] = int(data_split[3])
    reindeers.setdefault(reindeer_name, {})["max_travel_time"] = int(data_split[6])
    reindeers.setdefault(reindeer_name, {})["rest_time"] = int(data_split[-2])

In [5]:
schema = {
    "time": pl.Int64,
    "reindeer": str,
    "status": str,
    "time_travelled": pl.Int64,
    "rest_remaining": pl.Int64,
    "distance": pl.Int64,
}
time_range = pl.int_range(0, 100, eager=True).alias("time")
# print(time_range)

In [6]:
distance_reindeer_travelled = []
time_elapsed = 2503
for curr_reindeer in reindeers:
    df = pl.DataFrame(schema=schema)

    curr_status = "flying"
    rest_remaining = 0
    curr_time_travelled = 0
    curr_dist = 0

    reindeer_speed = reindeers[curr_reindeer]["speed"]
    max_travel_time = reindeers[curr_reindeer]["max_travel_time"]
    rest_time = reindeers[curr_reindeer]["rest_time"]

    for curr_time in range(1, time_elapsed+1):
        if curr_status == "flying":
            curr_dist += reindeer_speed
            curr_time_travelled += 1
            if curr_time_travelled >= max_travel_time:
                curr_status = "resting"
                rest_remaining = rest_time
        else:
            if rest_remaining > 0:
                rest_remaining -= 1
            if rest_remaining == 0 and curr_status == "resting":
                curr_status = "flying"
                curr_time_travelled = 0

        curr_instance = pl.DataFrame({
            "time": curr_time, 
            "reindeer": curr_reindeer, 
            "status": curr_status, 
            "time_travelled": curr_time_travelled, 
            "rest_remaining": rest_remaining, 
            "distance": curr_dist
        })
        df = pl.concat([df, curr_instance], how="vertical")
    last_row = df[-1]
    final_distance = df[-1]["distance"][0]
    distance_reindeer_travelled.append(final_distance)
    print(f"{curr_reindeer} travelled {final_distance} after {time_elapsed} seconds")
print(max(distance_reindeer_travelled))

Vixen travelled 2640 after 2503 seconds
Blitzen travelled 2496 after 2503 seconds
Rudolph travelled 2540 after 2503 seconds
Cupid travelled 2592 after 2503 seconds
Donner travelled 2655 after 2503 seconds
Dasher travelled 2460 after 2503 seconds
Comet travelled 2493 after 2503 seconds
Prancer travelled 2484 after 2503 seconds
Dancer travelled 2516 after 2503 seconds
2655


# part2

In [11]:
time_elapsed = 2503
rows = []
for curr_reindeer in reindeers:

    curr_status = "flying"
    rest_remaining = 0
    curr_time_travelled = 0
    curr_dist = 0

    reindeer_speed = reindeers[curr_reindeer]["speed"]
    max_travel_time = reindeers[curr_reindeer]["max_travel_time"]
    rest_time = reindeers[curr_reindeer]["rest_time"]

    for curr_time in range(1, time_elapsed+1):
        if curr_status == "flying":
            curr_dist += reindeer_speed
            curr_time_travelled += 1
            if curr_time_travelled >= max_travel_time:
                curr_status = "resting"
                rest_remaining = rest_time
        else:
            if rest_remaining > 0:
                rest_remaining -= 1
            if rest_remaining == 0 and curr_status == "resting":
                curr_status = "flying"
                curr_time_travelled = 0

        rows.append({
            "time": curr_time, 
            "reindeer": curr_reindeer, 
            "status": curr_status, 
            "time_travelled": curr_time_travelled, 
            "rest_remaining": rest_remaining, 
            "distance": curr_dist
        })
    print(f"Analysed for {curr_reindeer}")
df = pl.DataFrame(rows)
print(df)

Analysed for Vixen
Analysed for Blitzen
Analysed for Rudolph
Analysed for Cupid
Analysed for Donner
Analysed for Dasher
Analysed for Comet
Analysed for Prancer
Analysed for Dancer
shape: (22_527, 6)
┌──────┬──────────┬─────────┬────────────────┬────────────────┬──────────┐
│ time ┆ reindeer ┆ status  ┆ time_travelled ┆ rest_remaining ┆ distance │
│ ---  ┆ ---      ┆ ---     ┆ ---            ┆ ---            ┆ ---      │
│ i64  ┆ str      ┆ str     ┆ i64            ┆ i64            ┆ i64      │
╞══════╪══════════╪═════════╪════════════════╪════════════════╪══════════╡
│ 1    ┆ Vixen    ┆ flying  ┆ 1              ┆ 0              ┆ 8        │
│ 2    ┆ Vixen    ┆ flying  ┆ 2              ┆ 0              ┆ 16       │
│ 3    ┆ Vixen    ┆ flying  ┆ 3              ┆ 0              ┆ 24       │
│ 4    ┆ Vixen    ┆ flying  ┆ 4              ┆ 0              ┆ 32       │
│ 5    ┆ Vixen    ┆ flying  ┆ 5              ┆ 0              ┆ 40       │
│ …    ┆ …        ┆ …       ┆ …              ┆ …   

In [13]:
points_df = (
    df.with_columns(
        (pl.col("distance") == pl.col("distance").max().over("time")).cast(pl.Int64).alias("point")
    )
    .group_by("reindeer")
    .agg(pl.col("point").sum().alias("points"))
)

reindeer_points = {r["reindeer"]: r["points"] for r in points_df.to_dicts()}

In [15]:
print(reindeer_points)

{'Rudolph': 887, 'Dancer': 1, 'Blitzen': 5, 'Comet': 22, 'Donner': 414, 'Dasher': 0, 'Prancer': 153, 'Vixen': 1059, 'Cupid': 13}
